In [1]:
import sys
import os
os.chdir("/home/jonfrey/ASL")
sys.path.append("""/home/jonfrey/ASL/src/""")
sys.path.append("""/home/jonfrey/ASL/src/pseudo_label""")

import numpy as np
from visu import Visualizer
import imageio


# STD
import os
import copy

# MISC
import numpy as np
import torch
import imageio
import cv2
from PIL import Image

# matplotlib
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.backends.backend_agg import FigureCanvasAgg
import matplotlib.patches as patches

In [3]:
from PIL import Image
import cv2
import torch
from torchvision import transforms as tf
from utils_asl import LabelLoaderAuto

lla = LabelLoaderAuto(root_scannet="/home/jonfrey/Datasets/scannet")

In [4]:
from pathlib import Path
import copy
import numpy as np

class AccMonitor():
    def __init__(self):
        self.metrics = {}
        self.metric_dict = {'correct': 0,
                            'correct_valid': 0,
                            'correct_valid_both': 0,
                            'total': 0, 
                            'total_valid':0,
                            'total_valid_both':0,
                            'images': 0}
    def register(self,name):
        self.metrics[name] = copy.deepcopy(self.metric_dict)
        
    def update(self,name, label, gt):
        if name not in self.metrics.keys():
            self.register(name)
        m1 = gt != -1
        self.metrics[name]["total_valid"] += (m1).sum()
        self.metrics[name]["correct_valid"] += np.sum(gt[m1] == label[m1])
        
        m2 = (gt != -1 )*(label != -1)
        self.metrics[name]["total_valid_both"] += (m2).sum()
        self.metrics[name]["correct_valid_both"] += np.sum(gt[m2] == label[m2])
        
        self.metrics[name]["total"] += gt.size
        self.metrics[name]["correct"] += np.sum(gt == label)
        self.metrics[name]["images"] += 1
        
    def __str__(self):
        s = "Acc-Monitor:\n"
        for k in self.metrics.keys():
            acc = self.metrics[k]['correct_valid'] / self.metrics[k]['total_valid']
            acc2 = self.metrics[k]['correct_valid_both'] / self.metrics[k]['total_valid_both']
            s += f"  {k}: Nr-Images: {self.metrics[k]['images']} , Avg-Acc: {acc}  Avg-Acc: valid both {acc2} \n"  
        return s
        
accm = AccMonitor()
accm.update("Test", np.ones( (100,100)) ,np.ones( (100,100)))
accm.update("Test2", np.ones( (100,100)) ,np.ones( (100,100)))
print(accm)

Acc-Monitor:
  Test: Nr-Images: 1 , Avg-Acc: 1.0  Avg-Acc: valid both 1.0 
  Test2: Nr-Images: 1 , Avg-Acc: 1.0  Avg-Acc: valid both 1.0 



In [5]:
visu = Visualizer("/home/jonfrey/Documents/master_thesis/Report/images/regu", logger=None, epoch=0, store=False, num_classes=41)

def get_labels(p,sub=1,la="png"):
    p = [str(s) for s in Path(p).rglob(f'*.{la}') if str(s).find("_.png") == -1]
    
    p.sort(
        key=lambda x: int(x.split("/")[-1][:-4])
    )
    
    return p[::sub]

gt_p = get_labels("/home/jonfrey/Datasets/scannet/scans/scene0000_00/label-filt",sub=10)

img_p = get_labels("/home/jonfrey/Datasets/scannet/scans/scene0000_00/color",sub=10, la="jpg")


labels_desilation2_09 = get_labels("/home/jonfrey/Datasets/labels_generated/desilation2_09/scans/scene0000_00")
labels_52_retrained =  get_labels("/home/jonfrey/Datasets/labels_generated/labels_pretrain_retrain_52/scans/scene0000_00")
labels_52_retrained_05 = get_labels("/home/jonfrey/Datasets/labels_generated/labels_pretrain_retrain_52_05/scans/scene0000_00")
labels_52_repro = get_labels("/home/jonfrey/Datasets/labels_generated/labels_52/scans/scene0000_00")

In [ ]:
accm = AccMonitor()
for i in range(0, len(labels_52_repro)):
    gt = lla.get(gt_p[i])[0]
    l1 = lla.get(labels_52_repro[i])[0]
    l2 = lla.get(labels_desilation2_09[i])[0]
    l3 = lla.get(labels_52_retrained[i])[0] 
    l4 = lla.get(labels_52_retrained_05[i])[0] 
    
    accm.update("labels_52_repro", l1-1 ,gt-1)
    accm.update("labels_desilation2", l2-1 ,gt-1)
    accm.update("labels_52_retrained", l3-1 ,gt-1)
    accm.update("labels_pretrain_retrain_52_05", l4-1 ,gt-1)
    img = imageio.imread(img_p[i]) 
    if i == -1:
        visu.plot_detectron(np.array( imageio.imread(img_p[i]) ), l1, jupyter=True)
        visu.plot_detectron(np.array( imageio.imread(img_p[i]) ), l2, jupyter=True)
        visu.plot_detectron(np.array( imageio.imread(img_p[i]) ), l3, jupyter=True)
        visu.plot_detectron(np.array( imageio.imread(img_p[i]) ), l4, jupyter=True)
        visu.plot_detectron(np.array( imageio.imread(img_p[i]) ), gt, jupyter=True)

In [ ]:
visu = Visualizer("/home/jonfrey/Documents/master_thesis/Report/images/output_reprojected_labels", logger=None, epoch=0, store=True, num_classes=41)


i = 200
gt = lla.get(gt_p[i])[0]
l1 = lla.get(labels_52_repro[i])[0]
l2 = lla.get(labels_desilation2_09[i])[0]
l3 = lla.get(labels_52_retrained[i])[0] 
l4 = lla.get(labels_52_retrained_05[i])[0] 

img = imageio.imread(img_p[i]) 
visu.plot_detectron(np.array( imageio.imread(img_p[i]) ), l1, jupyter=True, tag="repro_iteration_1")
# visu.plot_detectron(np.array( imageio.imread(img_p[i]) ), l2, jupyter=True)
#visu.plot_detectron(np.array( imageio.imread(img_p[i]) ), l3, jupyter=True)
visu.plot_detectron(np.array( imageio.imread(img_p[i]) ), l4, jupyter=True, tag="retrained_iteration_2")
visu.plot_detectron(np.array( imageio.imread(img_p[i]) ), gt, jupyter=True, tag="gt")

In [ ]:
labels_iteration_2 = get_labels("/home/jonfrey/Datasets/labels_generated/labels_iteration_2/scans/scene0000_00")
accm2 = AccMonitor()
for i in range(0, len(labels_52_repro)):
    gt = lla.get(gt_p[i])[0]
    l1 = lla.get(labels_iteration_2[i])[0]
    
    accm2.update("labels_iteration2", l1-1 ,gt-1)
    print(i)
    if i == 0:
        visu.plot_detectron(np.array( imageio.imread(img_p[i]) ), l1, jupyter=True)
        visu.plot_detectron(np.array( imageio.imread(img_p[i]) ), gt, jupyter=True)

        img = imageio.imread(img_p[i]) 
        